## Chapter 14. Inheritance: For Better or for Worse

### The `super()` Function
When a subclass overrides a method of a superclass, the overriding method usually needs to call the corresponding method of the superclass.

Here's the recommended way


```python
class LastUpdatedOrderedDict(OrderedDict):
    """Store items in the order they were last updated"""

    def __setitem__(self, key, value):
        super().__setitem__(key, value)
```

You may call the method directly on the superclass, like below


```python
class NotRecommended(OrderedDict):
    """This is a counter example!"""

    def __setitem__(self, key, value):
        OrderedDict.__setitem__(self, key, value)
        self.move_to_end(key)
```

### Subclassing Built-In Types is Tricky

In [14]:
# Example 14-1. Our __setitem__ override is ignored by the __init__ and __update__ methods of the built-in dict

class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [ ]:
dd = DoppelDict(one=1)  # __init__ ignored that __setitem__ was overridden
print(dd)
dd['two'] = 2
print(dd)
dd.update(three=3)  # update method does not use the overriden __setitem__
dd

{'one': 1}
{'one': 1, 'two': [2, 2]}


{'one': 1, 'two': [2, 2], 'three': 3}

In [16]:
# Example 14-2. The __getitem__ of AnswerDict is bypassed by dict.update

class AnswerDict(dict):
    def __getitem__(self, key):
        return 42

In [ ]:
ad = AnswerDict(a='foo')
print(ad)
print(ad['a'])  # use the overriden __getitem__
d = {}
d.update(ad)
print(d['a'])
print(d)

{'a': 'foo'}
42
foo
{'a': 'foo'}


In [19]:
# Example 14-3. DoppelDict2 and AnswerDict2 work as expected because they extend UserDict and not dict

import collections

class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [20]:
dd = DoppelDict2(one=1)
print(dd)
dd['two'] = 2
print(dd)
dd.update(three=3)
print(dd)

{'one': [1, 1]}
{'one': [1, 1], 'two': [2, 2]}
{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}


In [21]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return 42

In [22]:
ad = AnswerDict2(a='foo')
print(ad['a'])
d = {}
d.update(ad)
print(d['a'])
d

42
42


{'a': 42}

### Multiple Inheritance and Method Resolution Order

In [23]:
# Example 14-4. diamond.py: classes Leaf, A, B, Root form the graph in Figure 14-1

class Root:  # <1>
    def ping(self):
        print(f'{self}.ping() in Root')

    def pong(self):
        print(f'{self}.pong() in Root')

    def __repr__(self):
        cls_name = type(self).__name__
        return f'<instance of {cls_name}>'


class A(Root):  # <2>
    def ping(self):
        print(f'{self}.ping() in A')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in A')
        super().pong()


class B(Root):  # <3>
    def ping(self):
        print(f'{self}.ping() in B')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in B')


class Leaf(A, B):  # <4>
    def ping(self):
        print(f'{self}.ping() in Leaf')
        super().ping()

In [ ]:
# Example 14-5. Doctests for calling ping and pong on a Leaf object

leaf1 = Leaf()
leaf1.ping()

<instance of Leaf>.ping() in Leaf
<instance of Leaf>.ping() in A
<instance of Leaf>.ping() in B
<instance of Leaf>.ping() in Root


In [25]:
leaf1.pong()

<instance of Leaf>.pong() in A
<instance of Leaf>.pong() in B


In [28]:
Leaf.__mro__ # doctest:+NORMALIZE_WHITESPACE

(__main__.Leaf, __main__.A, __main__.B, __main__.Root, object)

In [29]:
# Example 14-6. diamond2.py: classes to demonstrate the dynamic nature of super()

class U():  # <2>
    def ping(self):
        print(f'{self}.ping() in U')
        super().ping()  # <3>

class LeafUA(U, A):  # <4>
    def ping(self):
        print(f'{self}.ping() in LeafUA')
        super().ping()
# end::DIAMOND_CLASSES[]

class LeafAU(A, U):
    def ping(self):
        print(f'{self}.ping() in LeafAU')
        super().ping()

In [30]:
u = U()
u.ping()

<__main__.U object at 0x10871fe10>.ping() in U


AttributeError: 'super' object has no attribute 'ping'

In [31]:
leaf2 = LeafUA()
leaf2.ping()

<instance of LeafUA>.ping() in LeafUA
<instance of LeafUA>.ping() in U
<instance of LeafUA>.ping() in A
<instance of LeafUA>.ping() in Root


In [32]:
LeafUA.__mro__

(__main__.LeafUA, __main__.U, __main__.A, __main__.Root, object)

In [33]:
# Example 14-7. MRO of tkinter.Text
def print_mro(cls):
    print(', '.join(c.__name__ for c in cls.__mro__))

import tkinter
print_mro(tkinter.Text)

Text, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, YView, object


### Mixin Classes
A mixin class is designed to be subclassed together with at least one other class in a multiple inheritance arrangement.

In [2]:
# Example 14-8. uppermixin.py: UpperCaseMixin supports case-insensitive mappings

import collections

def _upper(key):  # <1>
    try:
        return key.upper()
    except AttributeError:
        return key

class UpperCaseMixin:  # <2>
    def __setitem__(self, key, item):
        super().__setitem__(_upper(key), item)

    def __getitem__(self, key):
        return super().__getitem__(_upper(key))

    def get(self, key, default=None):
        return super().get(_upper(key), default)

    def __contains__(self, key):
        return super().__contains__(_upper(key))


# Example 14-9. uppermixin.py: two classes that use UpperCaseMixin

class UpperDict(UpperCaseMixin, collections.UserDict):  # <1>
    pass

class UpperCounter(UpperCaseMixin, collections.Counter):  # <2>
    """Specialized 'Counter' that uppercases string keys"""  # <3>

In [3]:
d = UpperDict([('a', 'letter A'), (2, 'digit two')])
print(list(d.keys()))

d['b'] = 'letter B'
print('b' in d)

print(d['a'], d.get('B'))

print(list(d.keys()))


['A', 2]
True
letter A letter B
['A', 2, 'B']


In [4]:
c = UpperCounter('BaNanA')
print(c.most_common())

[('A', 3), ('N', 2), ('B', 1)]


In [ ]:
# Example 14-10. Part of Lib/socketserver.py in Python 3.10
class ThreadingMixIn:
    """Mixin class to handle each request in a new thread."""
    # 8 lines omitted in book listing

    def process_request_thread(self, request, client_address):
        ... # 6 lines omitted in book listing

    def process_request(self, request, client_address):
        ... # 8 lines omitted in book listing

    def server_close(self):
        super().server_close()
        self._threads.join()